In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PythonTeam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data=pd.read_csv('tweets.csv')

In [4]:
data=data[['tweet','class']]

In [5]:
def preprocess_text(text):
    text = str(text).lower()  
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  
    text = re.sub(r'\@\w+|\#', '', text)  
    text = re.sub(r'[^a-zA-Z\s]', '', text)  
    words = text.split()
    words = [word for word in words if word not in stop_words]  
    return " ".join(words)

In [6]:
data['clean_text']=data['tweet'].apply(preprocess_text)

In [7]:
x_train,x_test,y_train,y_test=train_test_split(data['clean_text'],data['class'],test_size=0.2,random_state=42)

In [8]:
vectorizer = TfidfVectorizer(max_features=5000)  
X_train_tfidf = vectorizer.fit_transform(x_train)
X_test_tfidf = vectorizer.transform(x_test)

In [9]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train_tfidf, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [10]:
y_pred = xgb_model.predict(X_test_tfidf)

In [11]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9016
Classification Report:
               precision    recall  f1-score   support

           0       0.52      0.22      0.31       290
           1       0.94      0.94      0.94      3832
           2       0.79      0.95      0.86       835

    accuracy                           0.90      4957
   macro avg       0.75      0.70      0.70      4957
weighted avg       0.89      0.90      0.89      4957



In [12]:
import pickle
pickle.dump(xgb_model,open('xgbmodel.pickle','wb'))

In [13]:
pickle.dump(vectorizer, open('TfidfVectorizer.pickle', 'wb'))